# Parámetros del Programa

In [1583]:
archivoVentas = 'ventasPrueba.csv'
archivoProd = 'prodPrueba.csv'

# NOTA: ESTE SCRIPT REQUIERE DE FECHAS DE PROGRAMACION PARA EL ARCHIVO DE VENTAS.
# NOTA: FALTA RIGHT JOIN PARA SACAR PROCEDIMIENTOS EFECTUADOS, PERO NO PRESUPUESTADOS
# FALTA ACTUALIZAR LA FECHA DE PROX CITA (FECHA_PRD + 30 O SIMILAR)
# AGREGAR BANDERA DE ANTECEDENTE PQR AL PACIENTE PARA QUE LO VEAN EN LA PROGRAMACION

# COLS: CLINICA-PTE-DNI-TTMTO-CANT_PROG-CANT_PEND-SEMANA_PROG-SEMANA_PROD-SEMANA_ACTUAL-SEMANAS_TRANSCURRIDAS
# LOS SIGUIENTES TTMTOS SIEMPRE DEBEN APARECER: MENS ORTO, CXGIAS (RASP, ELEV, REGEN, IMPLA, GINGI, EXO, FRENILL, ALARG), 
# PROTESIS



# Ajustes de Python

In [1584]:
import pandas as pd
import numpy as np

from IPython.display import display
pd.options.display.max_columns = None

pd.set_option("display.max_rows",1000)

# Carga de Datos

In [1585]:
venta_df = pd.read_csv(archivoVentas)
venta_df.head(1)

,idApunte,IdClinica,Nombre_Clinica,idContacto,nombre,apellidos,dni,Doctor,fechaApunte,Mes,Id_Tratamiento,Nombre_Tratamiento,Pieza,Cantidad,Precio,Precio_sin_descuento,Descuento_Porcentaje,Vr_descontado,Precio_con_descuento,TotalPagos
0,37436984,2000,Autopista 104A Bogota,826386,EVELYN,DAZA,111,LEIDIS MARIAN MEJIA CUENTAS,01/02/2018 8:49,February,16711,MENSUALIDAD ORTODONCIA METALICA,11,6,70000,1260000,17.9,225540,1034460,50000


In [1586]:
prod_df = pd.read_csv(archivoProd, engine='python')
prod_df.head(1)

,Clínica,Fecha Cita,Fecha Pago,DNI,Doctor,Paciente,Nº Historia,Presupuesto,Tratamiento,Categoría,Especialidad,Duración,Importe Tratamiento,% Descuento,Importe Descuento,Importe Producido,Equivalencia
0,Autopista 104A Bogota,02/01/2016,02/01/2016,111.0,MARIA ISABEL MUÑOZ QUINTERO,EVELYN DAZA,123.0,10.0,1x RETIRO BRACKETS ARCADA INFERIOR. Pieza: 11,Generalista,OPERATORIA GENERAL,30.0,90000.0,0%,0.0,90000.0,0.0


# Nuevas Columnas

In [1587]:
# agregar columnas nuevas a VENTA - DataFrame principal
#venta_df['ttmto_homol']='' 
venta_df['unids_pdtes']=None
venta_df['ultima_aten']=None
venta_df['fecha_prog']=None 
venta_df['retraso']=None
venta_df['estado']=None
venta_df['nivel_retraso']=None

# agregar columnas nuevas a PRODUCCION
prod_df['ttmto_homol']=''

# Homologaciones

In [1588]:
#homologacion de VENTAS

venta_df['ttmto_homol']=venta_df['Nombre_Tratamiento']

#diccionario de homologacion para VENTA y homologacion (si no esta incluida una homol, queda Nombre_Tratamiento)
di = {"RASPAJE Y PULIDO CORONAL(DETRARTAJE SUPRAGINGIVAL) X CUADRANTE": "RASPAJE_CUADRANTE", 
      "RETENEDOR FIJO 3X3 EN ALAMBRE": "RETENEDOR FIJO","RETENEDOR ESSIX O ESTETICOS": "RETENEDOR ESSIX", 
      "RETIRO BRACKETS ARCADA INFERIOR": "RETIRO BRACKETS ARCADA","RETIRO BRACKETS ARCADA SUPERIOR": "RETIRO BRACKETS ARCADA"}
venta_df['ttmto_homol'].replace(di, inplace=True)

# **uncomment to display venta_df:
#display(venta_df)


In [1589]:
#homologacion de PRODUCCION

prod_df['temp'] = prod_df['Tratamiento'].str.contains('MENSUALIDAD ORTODONCIA METALICA')
prod_df.loc[prod_df['temp'] == True,'ttmto_homol'] = 'MENSUALIDAD ORTODONCIA METALICA'

prod_df['temp'] = prod_df['Tratamiento'].str.contains('RETIRO BRACKETS ARCADA')
prod_df.loc[prod_df['temp'] == True,'ttmto_homol'] = 'RETIRO BRACKETS ARCADA'

prod_df['temp'] = prod_df['Tratamiento'].str.contains('RETIRO BRACKETS ARCADA')
prod_df.loc[prod_df['temp'] == True,'ttmto_homol'] = 'RETIRO BRACKETS ARCADA'

prod_df['temp'] = prod_df['Tratamiento'].str.contains('PROFILAXIS')
prod_df.loc[prod_df['temp'] == True,'ttmto_homol'] = 'PROFILAXIS'

prod_df['temp'] = prod_df['Tratamiento'].str.contains('DETARTRAJE')
prod_df.loc[prod_df['temp'] == True,'ttmto_homol'] = 'DETARTRAJE POR ARCADA'

prod_df['temp'] = prod_df['Tratamiento'].str.contains('ESSIX')
prod_df.loc[prod_df['temp'] == True,'ttmto_homol'] = 'RETENEDOR ESSIX'


# Preprocesamiento

In [1615]:
# NaN, vacíos,etc
# deshechar columnas innecesarias de ventas y produccion
prod_df = prod_df.dropna()
#prod_df

In [1618]:
# CAMBIAR FORMATO DE COLUMNA 'FECHA CITA' A DATE (SIN HORA) DE PROD
temp_series = pd.Series() 
temp_series = prod_df['Fecha Cita']
prod_df['Fecha Cita'] = pd.to_datetime(temp_series)

# concat nombre y apellido en VENTAS
venta_df['Paciente'] = venta_df['nombre'] + ' ' + venta_df['apellidos']

venta_df

,idApunte,IdClinica,Nombre_Clinica,idContacto,nombre,apellidos,dni,Doctor,fechaApunte,Mes,Id_Tratamiento,Nombre_Tratamiento,Pieza,Cantidad,Precio,Precio_sin_descuento,Descuento_Porcentaje,Vr_descontado,Precio_con_descuento,TotalPagos,unids_pdtes,ultima_aten,fecha_prog,retraso,estado,nivel_retraso,ttmto_homol,Paciente
0,37436984,2000,Autopista 104A Bogota,826386,EVELYN,DAZA,111,LEIDIS MARIAN MEJIA CUENTAS,01/02/2018 8:49,February,16711,MENSUALIDAD ORTODONCIA METALICA,11,6,70000,1260000,17.9,225540,1034460,50000,None,None,None,None,None,None,MENSUALIDAD ORTODONCIA METALICA,EVELYN DAZA
1,37771381,2000,Autopista 104A Bogota,826386,EVELYN,DAZA,111,MARIA APONTE MUNAR,08/02/2018 10:22,February,16711,MENSUALIDAD ORTODONCIA METALICA,11,3,70000,1260000,17.9,225540,1034460,16000,None,None,None,None,None,None,MENSUALIDAD ORTODONCIA METALICA,EVELYN DAZA
2,38147237,2000,Autopista 104A Bogota,826386,EVELYN,DAZA,111,LEIDIS MARIAN MEJIA CUENTAS,15/02/2018 9:44,February,16722,RETENEDOR FIJO 3X3 EN ALAMBRE,11,1,150000,150000,17.9,26850,123150,12650,None,None,None,None,None,None,RETENEDOR FIJO,EVELYN DAZA
3,38147242,2000,Autopista 104A Bogota,826386,EVELYN,DAZA,111,LEIDIS MARIAN MEJIA CUENTAS,15/02/2018 9:44,February,16721,RETENEDOR ESSIX O ESTETICOS,11,1,150000,150000,17.9,26850,123150,123150,None,None,None,None,None,None,RETENEDOR ESSIX,EVELYN DAZA
4,38147247,2000,Autopista 104A Bogota,826386,EVELYN,DAZA,111,LEIDIS MARIAN MEJIA CUENTAS,15/02/2018 9:44,February,16723,RETIRO BRACKETS ARCADA INFERIOR,31,1,100000,100000,17.9,17900,82100,82100,None,None,None,None,None,None,RETIRO BRACKETS ARCADA,EVELYN DAZA
5,38147252,2000,Autopista 104A Bogota,826386,EVELYN,DAZA,111,LEIDIS MARIAN MEJIA CUENTAS,15/02/2018 9:44,February,16724,RETIRO BRACKETS ARCADA SUPERIOR,11,1,100000,100000,17.9,17900,82100,82100,None,None,None,None,None,None,RETIRO BRACKETS ARCADA,EVELYN DAZA
6,38944406,2000,Autopista 104A Bogota,826386,EVELYN,DAZA,111,LEIDIS MARIAN MEJIA CUENTAS,27/02/2018 2:43,February,16722,RETENEDOR FIJO 3X3 EN ALAMBRE,11,1,150000,150000,17.9,26850,123150,110500,None,None,None,None,None,None,RETENEDOR FIJO,EVELYN DAZA
7,38944409,2000,Autopista 104A Bogota,B22,PEDRO,GOMEZ,222,LEIDIS MARIAN MEJIA CUENTAS,12/03/2018 2:43,March,16722,PROFILAXIS,11,1,400000,40000,0.0,0,40000,40000,None,None,None,None,None,None,PROFILAXIS,PEDRO GOMEZ
8,38944409,2000,Autopista 104A Bogota,B22,PEDRO,GOMEZ,222,LEIDIS MARIAN MEJIA CUENTAS,12/03/2018 2:43,March,16722,PROFILAXIS,11,1,400000,40000,0.0,0,40000,40000,None,None,None,None,None,None,PROFILAXIS,PEDRO GOMEZ
9,38944409,2000,Autopista 104A Bogota,B22,PEDRO,GOMEZ,222,LEIDIS MARIAN MEJIA CUENTAS,12/03/2018 2:43,March,16722,DETARTRAJE POR ARCADA,11,2,400000,40000,0.0,0,40000,40000,None,None,None,None,None,None,DETARTRAJE POR ARCADA,PEDRO GOMEZ


# Resumen de tratamientos por cantidades y Fecha de Prog

In [1617]:
# Resumen de CANTIDADES en Dataframe de VENTA
ventaResumen = venta_df.groupby(['Nombre_Clinica','Paciente','dni','ttmto_homol'], as_index = False).sum()
ventaResumen['fecha_prog'] = '' #esta fecha se llena en clinica y se va corriendo auto a medida que se evoluciona, en el cruce
ventaResumen

,Nombre_Clinica,Paciente,dni,ttmto_homol,idApunte,IdClinica,Id_Tratamiento,Pieza,Cantidad,Precio,Precio_sin_descuento,Descuento_Porcentaje,Vr_descontado,Precio_con_descuento,TotalPagos,fecha_prog
0,Autopista 104A Bogota,EVELYN DAZA,111,MENSUALIDAD ORTODONCIA METALICA,75208365,4000,33422,22,9,140000,2520000,35.8,451080,2068920,66000,
1,Autopista 104A Bogota,EVELYN DAZA,111,RETENEDOR ESSIX,38147242,2000,16721,11,1,150000,150000,17.9,26850,123150,123150,
2,Autopista 104A Bogota,EVELYN DAZA,111,RETENEDOR FIJO,77091643,4000,33444,22,2,300000,300000,35.8,53700,246300,123150,
3,Autopista 104A Bogota,EVELYN DAZA,111,RETIRO BRACKETS ARCADA,76294499,4000,33447,42,2,200000,200000,35.8,35800,164200,164200,
4,Autopista 104A Bogota,PEDRO GOMEZ,222,DETARTRAJE POR ARCADA,38944409,2000,16722,11,2,400000,40000,0.0,0,40000,40000,
5,Autopista 104A Bogota,PEDRO GOMEZ,222,PROFILAXIS,77888818,4000,33444,22,2,800000,80000,0.0,0,80000,80000,


In [1593]:
# Programar fecha de procedimientos (luego se exportará a csv y se diligenciara en excel)
temp_series = pd.Series()

temp_series = ['01/03/2018','01/03/2018','01/03/2018','01/03/2018','01/03/2018','01/03/2018']
ventaResumen['fecha_prog'] = pd.to_datetime(temp_series)
#ventaResumen

,Nombre_Clinica,Paciente,dni,ttmto_homol,idApunte,IdClinica,Id_Tratamiento,Pieza,Cantidad,Precio,Precio_sin_descuento,Descuento_Porcentaje,Vr_descontado,Precio_con_descuento,TotalPagos,fecha_prog
0,Autopista 104A Bogota,EVELYN DAZA,111,MENSUALIDAD ORTODONCIA METALICA,75208365,4000,33422,22,9,140000,2520000,35.8,451080,2068920,66000,2018-01-03
1,Autopista 104A Bogota,EVELYN DAZA,111,RETENEDOR ESSIX,38147242,2000,16721,11,1,150000,150000,17.9,26850,123150,123150,2018-01-03
2,Autopista 104A Bogota,EVELYN DAZA,111,RETENEDOR FIJO,77091643,4000,33444,22,2,300000,300000,35.8,53700,246300,123150,2018-01-03
3,Autopista 104A Bogota,EVELYN DAZA,111,RETIRO BRACKETS ARCADA,76294499,4000,33447,42,2,200000,200000,35.8,35800,164200,164200,2018-01-03
4,Autopista 104A Bogota,PEDRO GOMEZ,222,DETARTRAJE POR ARCADA,38944409,2000,16722,11,2,400000,40000,0.0,0,40000,40000,2018-01-03
5,Autopista 104A Bogota,PEDRO GOMEZ,222,PROFILAXIS,77888818,4000,33444,22,2,800000,80000,0.0,0,80000,80000,2018-01-03


In [1594]:
# Extraccion de Cantidad desde [Tratamientos] en PRODUCCION
prod_df['Cant'] = prod_df['Tratamiento'].astype(str).str[0]
prod_df = prod_df.astype({"Cant": int})
#prod_df

In [1595]:
prodResumen = prod_df.groupby(['Clínica','Paciente','DNI','ttmto_homol'], as_index = False).sum()
prodResumen = prodResumen.rename(index=str, columns={"Clínica": "Nombre_Clinica"})
prodResumen

,Nombre_Clinica,Paciente,DNI,ttmto_homol,Nº Historia,Presupuesto,Duración,Importe Tratamiento,Importe Descuento,Importe Producido,Equivalencia,Cant
0,Autopista 104A Bogota,EVELYN DAZA,111.0,MENSUALIDAD ORTODONCIA METALICA,492.0,20.0,120.0,280000.0,28000.0,252000.0,0.0,4
1,Autopista 104A Bogota,EVELYN DAZA,111.0,RETIRO BRACKETS ARCADA,123.0,10.0,30.0,90000.0,0.0,90000.0,0.0,1
2,Autopista 104A Bogota,PEDRO GOMEZ,222.0,DETARTRAJE POR ARCADA,123.0,5.0,30.0,70000.0,7000.0,63000.0,0.0,1
3,Autopista 104A Bogota,PEDRO GOMEZ,222.0,PROFILAXIS,369.0,15.0,90.0,210000.0,21000.0,189000.0,0.0,3


# Cruce de VENTAS y PRODUCCION y actualizacion de FECHA PROG

In [1596]:
# CONCIDENCIAS ENTRE VENTAS Y PROD
inner_df = pd.merge(ventaResumen, prodResumen, how='inner', left_on=['Nombre_Clinica','Paciente','dni','ttmto_homol'], right_on=['Nombre_Clinica','Paciente','DNI','ttmto_homol'])
# ACTUALIZACION DE CANTIDAD
inner_df['Cant_Pend'] = inner_df['Cantidad'] - inner_df['Cant']
inner_df = inner_df[['Nombre_Clinica','Paciente','dni','ttmto_homol','Cant_Pend','fecha_prog']] #el tel del paciente no está en los DFs. Solicitar
inner_df

,Nombre_Clinica,Paciente,dni,ttmto_homol,Cant_Pend,fecha_prog
0,Autopista 104A Bogota,EVELYN DAZA,111,MENSUALIDAD ORTODONCIA METALICA,5,2018-01-03
1,Autopista 104A Bogota,EVELYN DAZA,111,RETIRO BRACKETS ARCADA,1,2018-01-03
2,Autopista 104A Bogota,PEDRO GOMEZ,222,DETARTRAJE POR ARCADA,1,2018-01-03
3,Autopista 104A Bogota,PEDRO GOMEZ,222,PROFILAXIS,-1,2018-01-03


In [1597]:
# Actualización de la FECHA DE PROG para MENSUALIDAD ORTODONCIA METALICA
#inner_df['fecha_prog'] = inner_df['fecha_prog'] + pd.DateOffset(months=1)
boolMask = inner_df['ttmto_homol']=='MENSUALIDAD ORTODONCIA METALICA'
#temp_df = inner_df.where(boolMask).dropna()
#temp_df['fecha_prog']+ pd.DateOffset(months=1)

inner_df

,Nombre_Clinica,Paciente,dni,ttmto_homol,Cant_Pend,fecha_prog
0,Autopista 104A Bogota,EVELYN DAZA,111,MENSUALIDAD ORTODONCIA METALICA,5,2018-01-03
1,Autopista 104A Bogota,EVELYN DAZA,111,RETIRO BRACKETS ARCADA,1,2018-01-03
2,Autopista 104A Bogota,PEDRO GOMEZ,222,DETARTRAJE POR ARCADA,1,2018-01-03
3,Autopista 104A Bogota,PEDRO GOMEZ,222,PROFILAXIS,-1,2018-01-03


In [1598]:
# extraer procedimientos de VENTAS que no cruzaron con PROD
df1 = pd.merge(ventaResumen, prodResumen, how='left', left_on=['Nombre_Clinica','dni','ttmto_homol'], right_on=['Nombre_Clinica','DNI','ttmto_homol'])
df1 = df1[df1['Cant'].isnull()]
#df1

In [1599]:
#acomodar procedimientos que no cruzaron de la misma manera
df1 = df1.rename(index=str, columns={"Cantidad": "Cant_Pend"})
df1 = df1[['Nombre_Clinica','Paciente_x','dni','ttmto_homol','Cant_Pend','fecha_prog']]
df1 = df1.rename(index=str, columns={"Paciente_x": "Paciente"})
df1

,Nombre_Clinica,Paciente,dni,ttmto_homol,Cant_Pend,fecha_prog
1,Autopista 104A Bogota,EVELYN DAZA,111,RETENEDOR ESSIX,1,2018-01-03
2,Autopista 104A Bogota,EVELYN DAZA,111,RETENEDOR FIJO,2,2018-01-03


In [1602]:
#concatenar los procedimientos que no cruzaron con los que si cruzaron (PROGRAMA DE PRODUCCION)
ProdProg = pd.concat([df1, inner_df], ignore_index=True)
#ProdProg = ProdProg[['Nombre_Clinica','Paciente','dni','ttmto_homol','Cant_Pend','fecha_prog']]
ProdProg[ProdProg['fecha_prog'] < '2018-01-04']
#ProdProg[ProdProg['Cant_Pend']>0]
#venta_df

,Nombre_Clinica,Paciente,dni,ttmto_homol,Cant_Pend,fecha_prog
0,Autopista 104A Bogota,EVELYN DAZA,111,RETENEDOR ESSIX,1,2018-01-03
1,Autopista 104A Bogota,EVELYN DAZA,111,RETENEDOR FIJO,2,2018-01-03
2,Autopista 104A Bogota,EVELYN DAZA,111,MENSUALIDAD ORTODONCIA METALICA,5,2018-01-03
3,Autopista 104A Bogota,EVELYN DAZA,111,RETIRO BRACKETS ARCADA,1,2018-01-03
4,Autopista 104A Bogota,PEDRO GOMEZ,222,DETARTRAJE POR ARCADA,1,2018-01-03
5,Autopista 104A Bogota,PEDRO GOMEZ,222,PROFILAXIS,-1,2018-01-03


In [1614]:
ProdProg['fecha_prog']+ pd.DateOffset(days=14)

0   2018-01-17
1   2018-01-17
2   2018-01-17
3   2018-01-17
4   2018-01-17
5   2018-01-17
Name: fecha_prog, dtype: datetime64[ns]